In [1]:
import pandas as pd
import numpy as np
import streamlit as st

In [2]:
brasileirao = pd.read_excel('../data/brasileirao2023.xlsx')
brasileirao

,Temporada,Rodada,Mandante,Score_m,Visitante,Score_v
0,2023,1,PAL,2.0,CUI,1.0
1,2023,1,AME,0.0,FLU,3.0
2,2023,1,CAP,2.0,GOI,0.0
3,2023,1,BOT,2.0,SAO,1.0
4,2023,1,RED,2.0,BAH,1.0
...,...,...,...,...,...,...
375,2023,38,GOI,NaN,AME,NaN
376,2023,38,SAN,NaN,FOR,NaN
377,2023,38,SAO,NaN,FLA,NaN
378,2023,38,VAS,NaN,RED,NaN


In [3]:
brasileirao['Score_m']

0      2.0
1      0.0
2      2.0
3      2.0
4      2.0
      ... 
375    NaN
376    NaN
377    NaN
378    NaN
379    NaN
Name: Score_m, Length: 380, dtype: float64

In [4]:
brasileirao[brasileirao['Score_m'].notnull()]['Rodada'].count()

260

In [5]:
brasileirao = brasileirao[brasileirao['Score_m'].notnull()]
brasileirao

,Temporada,Rodada,Mandante,Score_m,Visitante,Score_v
0,2023,1,PAL,2.0,CUI,1.0
1,2023,1,AME,0.0,FLU,3.0
2,2023,1,CAP,2.0,GOI,0.0
3,2023,1,BOT,2.0,SAO,1.0
4,2023,1,RED,2.0,BAH,1.0
...,...,...,...,...,...,...
255,2023,26,CAM,1.0,CFC,2.0
256,2023,26,GOI,4.0,BAH,6.0
257,2023,26,COR,1.0,FLA,1.0
258,2023,26,PAL,1.0,SAN,2.0


In [6]:
times = brasileirao['Mandante'].unique()
tabela = pd.DataFrame()
for time in times:
    vit = emp = der = pro = con = jog = 0
    # emp = 0
    # der = 0
    # pro = 0
    # con = 0
    # jog = 0
    mandante = brasileirao[brasileirao['Mandante'] == time]
    for indice, partida in mandante.iterrows():
        if partida['Score_m'] > partida['Score_v']:
            vit += 1 
        elif partida['Score_m'] == partida['Score_v']:
            emp += 1
        else:
            der += 1
        pro += partida['Score_m']
        con += partida['Score_v']
        jog += 1
    visitante = brasileirao[brasileirao['Visitante'] == time]
    for indice, partida in visitante.iterrows():
        if partida['Score_v'] > partida['Score_m']:
            vit += 1
        elif partida['Score_v'] == partida['Score_m']:
            emp += 1
        else:
            der += 1
        pro += partida['Score_v']
        con += partida['Score_m']
        jog += 1
    new_row = pd.DataFrame({
        '1-Time': [time],
        '3-J' : [jog],
        '4-V' : [vit],
        '5-E' : [emp],
        '6-D' : [der],
        '7-GP': [pro],
        '8-GC': [con]
    })   
    tabela = pd.concat([tabela, new_row], ignore_index = True)
    
tabela['2-P'] = (tabela['4-V'] * 3) + tabela['5-E']
tabela['9-SG'] = tabela['7-GP'] - tabela['8-GC']   
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG
3,BOT,26,17,4,5,42.0,16.0,55,26.0
4,RED,26,12,10,4,36.0,23.0,46,13.0
9,GRE,26,13,5,8,42.0,35.0,44,7.0
0,PAL,26,12,8,6,40.0,22.0,44,18.0
8,FLA,26,12,8,6,38.0,31.0,44,7.0
5,FOR,26,12,6,8,33.0,26.0,42,7.0
10,FLU,26,12,5,9,34.0,31.0,41,3.0
2,CAP,26,11,8,7,38.0,30.0,41,8.0
6,CAM,26,11,7,8,30.0,21.0,40,9.0
12,SAO,26,9,8,9,31.0,27.0,35,4.0


In [7]:
tabela_sort[tabela_sort['7-GP'].notnull().sort_values(ascending=False)]['1-Time'].iloc[0]

C:\Users\samue\AppData\Local\Temp\ipykernel_13628\1631230441.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tabela_sort[tabela_sort['7-GP'].notnull().sort_values(ascending=False)]['1-Time'].iloc[0]


'BOT'

In [8]:
a = brasileirao[brasileirao['Score_m'].notnull()]['Rodada'].unique().max()
a

26

In [9]:
from sklearn.cluster import KMeans
df_data = tabela[[
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG'
]]
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_data)
tabela['cluster'] = kmeans.labels_
# tabela['c_p'] = kmeans.cluster_centers_[kmeans.labels_,0]
# tabela['c_v'] = kmeans.cluster_centers_[kmeans.labels_,1]
# tabela['c_e'] = kmeans.cluster_centers_[kmeans.labels_,2]
# tabela['c_d'] = kmeans.cluster_centers_[kmeans.labels_,3]
# tabela['c_sg'] = kmeans.cluster_centers_[kmeans.labels_,4]
tabela

c:\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
0,PAL,26,12,8,6,40.0,22.0,44,18.0,3
1,AME,26,4,6,16,29.0,54.0,18,-25.0,1
2,CAP,26,11,8,7,38.0,30.0,41,8.0,3
3,BOT,26,17,4,5,42.0,16.0,55,26.0,2
4,RED,26,12,10,4,36.0,23.0,46,13.0,3
5,FOR,26,12,6,8,33.0,26.0,42,7.0,3
6,CAM,26,11,7,8,30.0,21.0,40,9.0,3
7,COR,26,7,10,9,30.0,32.0,31,-2.0,0
8,FLA,26,12,8,6,38.0,31.0,44,7.0,3
9,GRE,26,13,5,8,42.0,35.0,44,7.0,3


In [10]:
print(kmeans.labels_)

[3 1 3 2 3 3 3 0 3 3 3 0 0 0 4 4 4 1 4 4]


In [11]:
kmeans.cluster_centers_

array([[ 32.5  ,   8.   ,   8.5  ,   9.5  ,   0.   ],
       [ 19.   ,   4.5  ,   5.5  ,  16.   , -25.   ],
       [ 55.   ,  17.   ,   4.   ,   5.   ,  26.   ],
       [ 42.75 ,  11.875,   7.125,   7.   ,   9.   ],
       [ 28.8  ,   7.2  ,   7.2  ,  11.6  ,  -9.6  ]])

In [12]:
df_cluster = pd.DataFrame()
for cluster, colunas in enumerate(kmeans.cluster_centers_):
    print(cluster)
    print(colunas)
    new_row = pd.DataFrame({
        'cluster': [cluster], 
        '2-P': [colunas[0]],              
        '4-V': [colunas[1]],
        '5-E': [colunas[2]],
        '6-D': [colunas[3]],
        '9-SG': [colunas[4]],
    })
    df_cluster = pd.concat([df_cluster, new_row], ignore_index=True)
df_cluster

0
[32.5  8.   8.5  9.5  0. ]
1
[ 19.    4.5   5.5  16.  -25. ]
2
[55. 17.  4.  5. 26.]
3
[42.75  11.875  7.125  7.     9.   ]
4
[28.8  7.2  7.2 11.6 -9.6]


,cluster,2-P,4-V,5-E,6-D,9-SG
0,0,32.50,8.000,8.500,9.5,0.0
1,1,19.00,4.500,5.500,16.0,-25.0
2,2,55.00,17.000,4.000,5.0,26.0
3,3,42.75,11.875,7.125,7.0,9.0
4,4,28.80,7.200,7.200,11.6,-9.6


In [13]:
df_cluster_sort = df_cluster.sort_values(by='2-P', ascending=False)
df_cluster_sort

,cluster,2-P,4-V,5-E,6-D,9-SG
2,2,55.00,17.000,4.000,5.0,26.0
3,3,42.75,11.875,7.125,7.0,9.0
0,0,32.50,8.000,8.500,9.5,0.0
4,4,28.80,7.200,7.200,11.6,-9.6
1,1,19.00,4.500,5.500,16.0,-25.0


In [14]:
df_cluster_sort['grupo'] = ['Título','Libertadores','Sul-Americana','Limbo','Rebaixamento']
df_cluster_sort

,cluster,2-P,4-V,5-E,6-D,9-SG,grupo
2,2,55.00,17.000,4.000,5.0,26.0,Título
3,3,42.75,11.875,7.125,7.0,9.0,Libertadores
0,0,32.50,8.000,8.500,9.5,0.0,Sul-Americana
4,4,28.80,7.200,7.200,11.6,-9.6,Limbo
1,1,19.00,4.500,5.500,16.0,-25.0,Rebaixamento


In [15]:
df_cluster_grupo = df_cluster_sort[['cluster','grupo']]
df_cluster_grupo

,cluster,grupo
2,2,Título
3,3,Libertadores
0,0,Sul-Americana
4,4,Limbo
1,1,Rebaixamento


In [16]:
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
0,PAL,26,12,8,6,40.0,22.0,44,18.0,3
1,AME,26,4,6,16,29.0,54.0,18,-25.0,1
2,CAP,26,11,8,7,38.0,30.0,41,8.0,3
3,BOT,26,17,4,5,42.0,16.0,55,26.0,2
4,RED,26,12,10,4,36.0,23.0,46,13.0,3
5,FOR,26,12,6,8,33.0,26.0,42,7.0,3
6,CAM,26,11,7,8,30.0,21.0,40,9.0,3
7,COR,26,7,10,9,30.0,32.0,31,-2.0,0
8,FLA,26,12,8,6,38.0,31.0,44,7.0,3
9,GRE,26,13,5,8,42.0,35.0,44,7.0,3


In [17]:
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort.style.background_gradient(cmap='Blues')
# tabela_sort.style.background_gradient(cmap='Blues').format("{:.2f}").render()


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
3,BOT,26,17,4,5,42.000000,16.000000,55,26.000000,2
4,RED,26,12,10,4,36.000000,23.000000,46,13.000000,3
9,GRE,26,13,5,8,42.000000,35.000000,44,7.000000,3
0,PAL,26,12,8,6,40.000000,22.000000,44,18.000000,3
8,FLA,26,12,8,6,38.000000,31.000000,44,7.000000,3
5,FOR,26,12,6,8,33.000000,26.000000,42,7.000000,3
10,FLU,26,12,5,9,34.000000,31.000000,41,3.000000,3
2,CAP,26,11,8,7,38.000000,30.000000,41,8.000000,3
6,CAM,26,11,7,8,30.000000,21.000000,40,9.000000,3
12,SAO,26,9,8,9,31.000000,27.000000,35,4.000000,0


In [18]:
#NORMALIZAR A TABELA E COLORIR...

#grafico com pontuacao original e dos cluster

#grafico dos cluster

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


In [19]:
df_data = tabela[[
    'cluster',              
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG'
]]

In [20]:
X_Train = df_data.drop(columns=['cluster'], axis=1)
X_Test = df_data.drop(columns=['cluster'], axis=1)
y_Train = df_data['cluster']
y_Test = df_data['cluster']

In [21]:
sc_x = StandardScaler()
X_Train = sc_x.fit_transform(X_Train)
X_Test = sc_x.fit_transform(X_Test)

In [22]:
logreg = LogisticRegression(solver="lbfgs", max_iter=500)
logreg.fit(X_Train, y_Train)
pred_logreg = logreg.predict(X_Test)
pred_proba = logreg.predict_proba(X_Test)

In [23]:
tabela["cluster_pred"] = pred_logreg

In [24]:
lista_proba = pred_proba.tolist()

data = []  # Lista para armazenar os dicionários de dados

index = 0
for proba in lista_proba:
    for i in range(0, len(proba)):
        new_row = {"index": index, "prob": i, "valor": round(proba[i], 4)}
        data.append(new_row)
    index += 1

df_prob = pd.DataFrame(data)  # Cria o DataFrame a partir da lista de dicionários
df_prob = df_prob.pivot_table(
    index="index", columns="prob", values="valor", aggfunc="sum"
)
df_prob = df_prob.reset_index()
df_prob = df_prob.set_index("index")


In [25]:
df_prob = df_prob.rename(columns={
    0.0: 'cl_o',
    1.0: 'cl_1',
    2.0: 'cl_2',
    3.0: 'cl_3',
    4.0: 'cl_4'
})

In [26]:
tabela = pd.merge(tabela, df_prob, left_index=True, right_index=True)
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,26,12,8,6,40.0,22.0,44,18.0,3,3,0.0973,0.0001,0.0483,0.8388,0.0154
1,AME,26,4,6,16,29.0,54.0,18,-25.0,1,1,0.0238,0.6428,0.0000,0.0006,0.3328
2,CAP,26,11,8,7,38.0,30.0,41,8.0,3,3,0.1750,0.0008,0.0262,0.7371,0.0608
3,BOT,26,17,4,5,42.0,16.0,55,26.0,2,3,0.0013,0.0000,0.4788,0.5196,0.0003
4,RED,26,12,10,4,36.0,23.0,46,13.0,3,3,0.1005,0.0000,0.0152,0.8734,0.0109
5,FOR,26,12,6,8,33.0,26.0,42,7.0,3,3,0.0844,0.0017,0.0792,0.7766,0.0580
6,CAM,26,11,7,8,30.0,21.0,40,9.0,3,3,0.1679,0.0019,0.0432,0.7089,0.0781
7,COR,26,7,10,9,30.0,32.0,31,-2.0,0,0,0.5783,0.0085,0.0007,0.1191,0.2934
8,FLA,26,12,8,6,38.0,31.0,44,7.0,3,3,0.0931,0.0003,0.0317,0.8439,0.0312
9,GRE,26,13,5,8,42.0,35.0,44,7.0,3,3,0.0393,0.0012,0.1440,0.7806,0.0349


In [27]:
df_pred = tabela[[
    '1-Time',
    'cl_o',
    'cl_1',
    'cl_2',
    'cl_3',
    'cl_4',
]]
df_pred = df_pred.copy()
df_pred

,1-Time,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,0.0973,0.0001,0.0483,0.8388,0.0154
1,AME,0.0238,0.6428,0.0000,0.0006,0.3328
2,CAP,0.1750,0.0008,0.0262,0.7371,0.0608
3,BOT,0.0013,0.0000,0.4788,0.5196,0.0003
4,RED,0.1005,0.0000,0.0152,0.8734,0.0109
5,FOR,0.0844,0.0017,0.0792,0.7766,0.0580
6,CAM,0.1679,0.0019,0.0432,0.7089,0.0781
7,COR,0.5783,0.0085,0.0007,0.1191,0.2934
8,FLA,0.0931,0.0003,0.0317,0.8439,0.0312
9,GRE,0.0393,0.0012,0.1440,0.7806,0.0349


In [28]:
df_pred.style.background_gradient(cmap='Blues', axis=1)

,1-Time,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,0.097300,0.000100,0.048300,0.838800,0.015400
1,AME,0.023800,0.642800,0.000000,0.000600,0.332800
2,CAP,0.175000,0.000800,0.026200,0.737100,0.060800
3,BOT,0.001300,0.000000,0.478800,0.519600,0.000300
4,RED,0.100500,0.000000,0.015200,0.873400,0.010900
5,FOR,0.084400,0.001700,0.079200,0.776600,0.058000
6,CAM,0.167900,0.001900,0.043200,0.708900,0.078100
7,COR,0.578300,0.008500,0.000700,0.119100,0.293400
8,FLA,0.093100,0.000300,0.031700,0.843900,0.031200
9,GRE,0.039300,0.001200,0.144000,0.780600,0.034900


In [29]:
rodadas = brasileirao['Rodada'].unique()
times = brasileirao['Mandante'].unique()
pontuacao = pd.DataFrame()
data = []  # Lista para armazenar os dicionários de dados

for rodada in rodadas:
    for time in times:
        resultado = brasileirao[(brasileirao['Rodada'] == rodada) & ((brasileirao['Mandante'] == time) | (brasileirao['Visitante'] == time))]
        if len(resultado) > 0:
            resultado = resultado.reset_index()
            if resultado['Mandante'][0] == time:
                if resultado['Score_m'][0] > resultado['Score_v'][0]:
                    pontos = 3
                elif resultado['Score_m'][0] == resultado['Score_v'][0]:
                    pontos = 1
                else:
                    pontos = 0
            else:                  
                if resultado['Score_v'][0] > resultado['Score_m'][0]:
                    pontos = 3
                elif resultado['Score_v'][0] == resultado['Score_m'][0]:
                    pontos = 1
                else:
                    pontos = 0
            new_row = {
                'time': time,
                'rodada': rodada,
                'pontos': pontos
            }        
            data.append(new_row)  # Adiciona o dicionário à lista de dados

pontuacao = pd.DataFrame(data)  # Cria o DataFrame a partir da lista de dicionários

pt_pontuacao = pontuacao.pivot_table(index='rodada', columns='time', values='pontos', aggfunc='sum')
pt_pontuacao_cum = pt_pontuacao.cumsum()
pt_pontuacao_cum

time,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,FLA,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
rodada,,,,,,,,,,,,,,,,,,,,
1,0,0,3,0,3,0,3,0,0,3,3,1,0,3,1,3,3,0,0,3
2,0,0,6,1,3,0,3,3,1,3,6,4,3,3,4,4,4,1,3,4
3,0,3,9,4,3,1,3,6,1,3,6,7,3,6,7,7,4,4,4,4
4,0,6,12,4,6,1,4,9,4,3,7,8,3,7,7,10,5,4,7,5
5,1,6,15,7,9,2,4,9,4,6,10,9,3,7,7,13,6,7,8,6
6,1,6,15,10,12,2,5,12,4,9,13,10,6,8,7,14,7,10,9,6
7,4,7,18,13,12,2,5,12,7,12,13,10,7,11,7,15,10,11,12,6
8,4,7,21,14,12,3,8,13,8,13,13,13,7,14,10,16,13,11,15,6
9,7,8,21,17,15,3,8,13,11,16,16,14,7,17,11,19,13,12,15,6


In [30]:
pt_pontuacao_cum = pt_pontuacao_cum.reset_index()
pt_pontuacao_cum

time,rodada,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,...,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
0,1,0,0,3,0,3,0,3,0,0,...,3,1,0,3,1,3,3,0,0,3
1,2,0,0,6,1,3,0,3,3,1,...,6,4,3,3,4,4,4,1,3,4
2,3,0,3,9,4,3,1,3,6,1,...,6,7,3,6,7,7,4,4,4,4
3,4,0,6,12,4,6,1,4,9,4,...,7,8,3,7,7,10,5,4,7,5
4,5,1,6,15,7,9,2,4,9,4,...,10,9,3,7,7,13,6,7,8,6
5,6,1,6,15,10,12,2,5,12,4,...,13,10,6,8,7,14,7,10,9,6
6,7,4,7,18,13,12,2,5,12,7,...,13,10,7,11,7,15,10,11,12,6
7,8,4,7,21,14,12,3,8,13,8,...,13,13,7,14,10,16,13,11,15,6
8,9,7,8,21,17,15,3,8,13,11,...,16,14,7,17,11,19,13,12,15,6
9,10,8,9,24,18,16,4,9,14,12,...,17,14,8,17,14,22,14,13,15,6


In [31]:
from sklearn.linear_model import LinearRegression

In [32]:
colunas = pt_pontuacao_cum.columns
df_regressao = pd.DataFrame()
data_regressao = []  # Lista para armazenar os dicionários de resultados de regressão

for coluna in colunas:
    if coluna != 'rodada':
        X = pt_pontuacao_cum['rodada'].values.reshape(-1, 1)
        y = pt_pontuacao_cum[coluna].values.reshape(-1, 1)
        regressor = LinearRegression()
        regressor.fit(X, y)
        A = regressor.intercept_[0]
        B = regressor.coef_[0][0]
        x = A + (B * 38)
        new_row_regressao = {
            'time': coluna,
            'intercept': round(A, 2),
            'slope': round(B, 2),
            'pontuacao_final': round(x, 2)
        }
        data_regressao.append(new_row_regressao)

df_regressao = pd.DataFrame(data_regressao)
df_regressao.sort_values(by='pontuacao_final', ascending=False)

,time,intercept,slope,pontuacao_final
2,BOT,3.57,2.13,84.68
13,GRE,0.09,1.77,67.29
15,PAL,2.52,1.69,66.55
9,FLA,-0.58,1.76,66.45
16,RED,-1.77,1.78,65.79
10,FLU,1.61,1.58,61.60
4,CAP,0.08,1.58,60.26
11,FOR,0.77,1.47,56.52
3,CAM,0.15,1.47,55.92
18,SAO,1.95,1.32,52.10


In [33]:
df_regressao = df_regressao[['time','slope']]
df_regressao

,time,slope
0,AME,0.74
1,BAH,1.04
2,BOT,2.13
3,CAM,1.47
4,CAP,1.58
5,CFC,0.79
6,COR,1.27
7,CRU,1.11
8,CUI,1.43
9,FLA,1.76


In [34]:
tabela = tabela.merge(df_regressao, left_on='1-Time', right_on='time', how='left')
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4,time,slope
0,PAL,26,12,8,6,40.0,22.0,44,18.0,3,3,0.0973,0.0001,0.0483,0.8388,0.0154,PAL,1.69
1,AME,26,4,6,16,29.0,54.0,18,-25.0,1,1,0.0238,0.6428,0.0000,0.0006,0.3328,AME,0.74
2,CAP,26,11,8,7,38.0,30.0,41,8.0,3,3,0.1750,0.0008,0.0262,0.7371,0.0608,CAP,1.58
3,BOT,26,17,4,5,42.0,16.0,55,26.0,2,3,0.0013,0.0000,0.4788,0.5196,0.0003,BOT,2.13
4,RED,26,12,10,4,36.0,23.0,46,13.0,3,3,0.1005,0.0000,0.0152,0.8734,0.0109,RED,1.78
5,FOR,26,12,6,8,33.0,26.0,42,7.0,3,3,0.0844,0.0017,0.0792,0.7766,0.0580,FOR,1.47
6,CAM,26,11,7,8,30.0,21.0,40,9.0,3,3,0.1679,0.0019,0.0432,0.7089,0.0781,CAM,1.47
7,COR,26,7,10,9,30.0,32.0,31,-2.0,0,0,0.5783,0.0085,0.0007,0.1191,0.2934,COR,1.27
8,FLA,26,12,8,6,38.0,31.0,44,7.0,3,3,0.0931,0.0003,0.0317,0.8439,0.0312,FLA,1.76
9,GRE,26,13,5,8,42.0,35.0,44,7.0,3,3,0.0393,0.0012,0.1440,0.7806,0.0349,GRE,1.77


In [35]:
from sklearn.cluster import KMeans
df_data = tabela[[
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG',
    'slope'
]]
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_data)
tabela['cluster'] = kmeans.labels_
tabela['c_p'] = kmeans.cluster_centers_[kmeans.labels_,0]
tabela['c_v'] = kmeans.cluster_centers_[kmeans.labels_,1]
tabela['c_e'] = kmeans.cluster_centers_[kmeans.labels_,2]
tabela['c_d'] = kmeans.cluster_centers_[kmeans.labels_,3]
tabela['c_sg'] = kmeans.cluster_centers_[kmeans.labels_,4]
tabela['c_slope'] = kmeans.cluster_centers_[kmeans.labels_,5]
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort.style.background_gradient(cmap='Blues')


c:\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4,time,slope,c_p,c_v,c_e,c_d,c_sg,c_slope
3,BOT,26,17,4,5,42.000000,16.000000,55,26.000000,2,3,0.001300,0.000000,0.478800,0.519600,0.000300,BOT,2.130000,55.000000,17.000000,4.000000,5.000000,26.000000,2.130000
4,RED,26,12,10,4,36.000000,23.000000,46,13.000000,3,3,0.100500,0.000000,0.015200,0.873400,0.010900,RED,1.780000,42.750000,11.875000,7.125000,7.000000,9.000000,1.637500
9,GRE,26,13,5,8,42.000000,35.000000,44,7.000000,3,3,0.039300,0.001200,0.144000,0.780600,0.034900,GRE,1.770000,42.750000,11.875000,7.125000,7.000000,9.000000,1.637500
0,PAL,26,12,8,6,40.000000,22.000000,44,18.000000,3,3,0.097300,0.000100,0.048300,0.838800,0.015400,PAL,1.690000,42.750000,11.875000,7.125000,7.000000,9.000000,1.637500
8,FLA,26,12,8,6,38.000000,31.000000,44,7.000000,3,3,0.093100,0.000300,0.031700,0.843900,0.031200,FLA,1.760000,42.750000,11.875000,7.125000,7.000000,9.000000,1.637500
5,FOR,26,12,6,8,33.000000,26.000000,42,7.000000,3,3,0.084400,0.001700,0.079200,0.776600,0.058000,FOR,1.470000,42.750000,11.875000,7.125000,7.000000,9.000000,1.637500
10,FLU,26,12,5,9,34.000000,31.000000,41,3.000000,3,3,0.075900,0.005800,0.103200,0.716800,0.098200,FLU,1.580000,42.750000,11.875000,7.125000,7.000000,9.000000,1.637500
2,CAP,26,11,8,7,38.000000,30.000000,41,8.000000,3,3,0.175000,0.000800,0.026200,0.737100,0.060800,CAP,1.580000,42.750000,11.875000,7.125000,7.000000,9.000000,1.637500
6,CAM,26,11,7,8,30.000000,21.000000,40,9.000000,3,3,0.167900,0.001900,0.043200,0.708900,0.078100,CAM,1.470000,42.750000,11.875000,7.125000,7.000000,9.000000,1.637500
12,SAO,26,9,8,9,31.000000,27.000000,35,4.000000,0,0,0.393800,0.007800,0.009300,0.367800,0.221300,SAO,1.320000,32.500000,8.000000,8.500000,9.500000,0.000000,1.282500


In [36]:
pontuacao_2 = pt_pontuacao_cum[pt_pontuacao_cum['rodada'] > 19]
pontuacao_2

time,rodada,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,...,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
19,20,10,21,48,27,32,14,25,25,28,...,34,29,23,34,24,37,32,21,28,19
20,21,13,21,51,30,33,14,26,25,28,...,35,32,24,37,25,40,35,21,28,19
21,22,16,22,51,31,34,14,27,26,28,...,38,32,25,40,26,41,36,21,31,20
22,23,17,25,51,34,37,14,27,29,29,...,38,35,25,40,29,44,39,21,31,23
23,24,17,25,51,37,40,14,30,29,29,...,41,38,26,43,29,44,42,24,31,26
24,25,18,25,52,40,40,17,30,30,32,...,41,39,27,44,29,44,45,27,34,26
25,26,18,28,55,40,41,20,31,31,33,...,41,42,27,44,32,44,46,30,35,27


In [37]:
colunas = pontuacao_2.columns
df_regressao = pd.DataFrame()
colunas = pontuacao_2.columns
regressao_data = []

for coluna in colunas:
    if coluna != 'rodada':
        X = pontuacao_2['rodada'].values.reshape(-1, 1)
        y = pontuacao_2[coluna].values.reshape(-1, 1)
        regressor = LinearRegression()
        regressor.fit(X, y)
        A = regressor.intercept_[0]
        B = regressor.coef_[0][0]
        x = A + (B * 38)
        regressao_data.append({
            'time': coluna,
            'intercept': round(A, 2),
            'slope': round(B, 2),
            'pontuacao_final': round(x, 2)
        })

df_regressao = pd.DataFrame(regressao_data)
df_regressao.sort_values(by='pontuacao_final', ascending=False)

,time,intercept,slope,pontuacao_final
16,RED,-16.57,2.43,75.71
3,CAM,-19.25,2.32,68.96
11,FOR,-13.18,2.11,66.89
13,GRE,1.68,1.68,65.46
2,BOT,32.39,0.82,63.61
9,FLA,4.93,1.50,61.93
4,CAP,-1.89,1.68,61.89
15,PAL,15.71,1.14,59.14
10,FLU,8.71,1.29,57.57
18,SAO,4.04,1.18,48.82
